In [1]:
import os
import sys
from pyspark.sql import *
from pyspark.sql.types import StructType, StructField, DateType, StringType, IntegerType
from pyspark.sql.functions import spark_partition_id

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:

spark = SparkSession.builder \
                    .appName("Data Sink Demo") \
                    .master("local[3]") \
                    .getOrCreate()

In [3]:
parquet_df = spark.read \
                .format("parquet") \
                .load("flight-time.parquet")
parquet_df.show()

Py4JJavaError: An error occurred while calling o33.load.
: java.util.ServiceConfigurationError: org.apache.spark.sql.sources.DataSourceRegister: Provider org.apache.spark.sql.avro.AvroFileFormat could not be instantiated
	at java.base/java.util.ServiceLoader.fail(ServiceLoader.java:586)
	at java.base/java.util.ServiceLoader$ProviderImpl.newInstance(ServiceLoader.java:813)
	at java.base/java.util.ServiceLoader$ProviderImpl.get(ServiceLoader.java:729)
	at java.base/java.util.ServiceLoader$3.next(ServiceLoader.java:1403)
	at scala.collection.convert.Wrappers$JIteratorWrapper.next(Wrappers.scala:46)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at s

In [ ]:
# write the df into avro
parquet_df.write \
    .format("csv") \
    .mode("overwrite") \
    .option("path", "data-writer/") \
    .save()

In [ ]:
print("num partition before " + str(parquet_df.rdd.getNumPartitions()))

num partition before 2


In [ ]:
partition_df = parquet_df.repartition(5)

NameError: name 'parquet_df' is not defined

In [ ]:
# write the df into avro
partition_df.write \
    .format("csv") \
    .mode("overwrite") \
    .option("path", "data-writer/csv/") \
    .save()

In [ ]:
# writing into json
parquet_df.write \
        .format("json") \
        .mode("overwrite") \
        .option("path", "data-writer/json/") \
        .partitionBy("OP_CARRIER", "ORIGIN") \
        .option("maxRecordsPerFile", 10000) \
        .save()